In [ ]:
import egglib
import glob 
import pandas as pd
import numpy as np
import os
import ipyparallel as ipp
import matplotlib.pyplot as plt
import time
from scipy.stats import pearsonr

In [ ]:
print(egglib.__version__)
print(ipp.__version__)

Run egglib on VCFs containing invariant and variant sites from accessions used to create NLRome

In [ ]:
#save file names in chromosome order
directory='/global/scratch/users/chandlersutherland/e14/popgen/vcf_1001_full/'
files = glob.glob(os.path.join(directory, "nlrome_invar_*_cds.recode.vcf"))
files.sort()

In [ ]:
#load the CDS bed file 
bed=pd.read_csv('/global/scratch/users/chandlersutherland/e14/popgen/vcf_1001_full/cds.bed', index_col=0)

In [ ]:
#define a fucntion that takes the vcf path, creates a vcf object and an index, then returns the egglib vcf object 
def importer(vcf_path):
    vcf=egglib.io.VcfParser(vcf_path)
    egglib.io.make_vcf_index(vcf_path)
    vcf.load_index(vcf_path+'i')
    return vcf

vcf1=importer(files[0])
vcf2=importer(files[1])
vcf3=importer(files[2])
vcf4=importer(files[3])
vcf5=importer(files[4])

In [ ]:
#iterate through the file and see if egglib errors on any problematic lines
#only have to do this once 
def parse_check(vcf, chrom):
    vcf.goto(chrom, position=egglib.io.FIRST)
    ln=0
    for i in vcf:
        ln += 1
        try: 
            chrom, pos, nall=next(vcf)
            #print(chrom, pos, nall)
        except:
            print('error line', ln)
            print(chrom, pos)
            #next(vcf)
    return print('checked')

print(files[0])
parse_check(vcf1, '1')
print(files[1])
parse_check(vcf2, '2')
print(files[2])
parse_check(vcf3, '3')
print(files[3])
parse_check(vcf4, '4')
print(files[4])
parse_check(vcf5, '5')

In [ ]:
#function that takes chromosome, CDS start, and CDS end, then returns an array with the vcf sites in that zone
def exon_search(bed_chrom, bed_start, bed_end, vcf):
    array=[]
    for i in vcf:
        chrom, pos, nall = next(vcf)
        if int(chrom) == bed_chrom:
            if pos >= bed_start and pos <= bed_end:
                site=vcf.get_genotypes()
                array.append(site)
            if pos > bed_end:
                break
    return array

In [ ]:
#takes in CDS, harvests sites for each piece of the CDS, then calculate over it 
def combine_cds(one_gene, vcf):
    cs = egglib.stats.ComputeStats()
    cs.configure(multi_hits=True, multi=False)
    cs.add_stats('Pi', 'lseff', 'D')

    sites=[]
    cds_length=0
    for index, row in one_gene.iterrows():
        bed_chrom=int(row['chrom'])
        bed_start=int(row['bedStart'])
        bed_end=int(row['bedEnd'])
        cds_length += bed_end-bed_start
        results=exon_search(bed_chrom, bed_start, bed_end, vcf)
        sites.extend(results)

    stats=cs.process_sites(sites)
    
    lseff=stats['lseff']
    D=stats['D']
    if pd.isnull(stats['D']):
        D=np.nan
    Pi=stats['Pi']
        
    gene=one_gene.iloc[0,3]
    result={'gene':gene, 'Pi_raw':Pi, 'D':D, 'lseff':lseff, 'cds_length':cds_length}
    return result

In [ ]:
#takes in chromosome integer, performs calculations, and outputs 
def wrapper(i):
    vcf=importer(files[i-1]) #import vcf file (convert to 0 integer indexing)
    genes=bed[bed['chrom']==i]['name'].unique() #filter gene list 
    bed_g=bed.groupby(['name'])
    stats=pd.DataFrame()
    vcf.goto(str(i), position=egglib.io.FIRST)
    for j in genes:
       sub=bed_g.get_group(j)
       result=combine_cds(sub, vcf)
       stats=stats.append(result, ignore_index=True)
    return stats

This is going to take a long time, and since everything is split by chromosome let's try running in parallel.

In [ ]:
#first create client to interact with cluster 
my_cluster=ipp.Cluster(n=5)
c=my_cluster.start_and_connect_sync()
c.ids

In [ ]:
#create direct view objects 
dview=c[:]
dview.block=True

In [ ]:
#load necessary packages in worker processes 
dview.execute('import pandas as pd')
dview.execute('import egglib')
dview.execute('import numpy as np')

In [ ]:
#this chunk runs the calculation, takes forever!!
mydict=dict(bed=bed, files=files, combine_cds=combine_cds, exon_search=exon_search, importer=importer)
dview.push(mydict)
res=dview.map(wrapper, range(1,6))
res

In [ ]:
#concatenate dataframes and save the result 
stats=pd.concat(res)
stats['Pi_by_dna']=stats['Pi_raw']/stats['cds_length']
stats['Pi_by_lseff']=stats['Pi_raw']/stats['lseff']
stats.to_csv('/global/scratch/users/chandlersutherland/e14/popgen/vcf_1001_full/cds_egglib_all.csv')